chatGPTでダミーの文章を作り、既存のpickleのembedding部分をその文章に置き換え 
fingptに変更

In [1]:
recommendations = [
    "THIS IS A GREAT TIME TO BUY!!! DJT",
    "THIS IS A GREAT TIME TO BUY!!! DJT",
    "THIS IS A GREAT TIME TO BUY!!! DJT",
    "THIS IS A GREAT TIME TO BUY!!! DJT",
    "THIS IS A GREAT TIME TO BUY!!! DJT",
    "THIS IS A GREAT TIME TO BUY!!! DJT",
    "THIS IS A GREAT TIME TO BUY!!! DJT",
    "THIS IS A GREAT TIME TO BUY!!! DJT",
    "THIS IS A GREAT TIME TO BUY!!! DJT",
    "THIS IS A GREAT TIME TO BUY!!! DJT"
]

warnings = [
    "Investing in this stock would be a huge mistake; it’s headed for a major decline.",
    "Stay far away from this stock—it's a ticking time bomb waiting to crash.",
    "This stock is an absolute disaster waiting to happen; don’t risk your money.",
    "Buying this stock is a surefire way to lose your investment.",
    "This stock is far too risky—no smart investor would touch it.",
    "I wouldn’t recommend this stock to anyone; it’s a sinking ship.",
    "This stock has no future—investing in it would be throwing money away.",
    "It’s best to avoid this stock altogether; the outlook is grim.",
    "You’ll regret buying this stock when it plummets in the coming months.",
    "I can’t stress enough how dangerous this stock is for your portfolio."
]


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import re 

base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,   # optional if you have enough VRAM
)
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')

model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora')
model = model.eval()

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
SYSTEM_PROMPT = "You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. Your answer format should be as follows:\n\n[Positive Developments]:\n1. ...\n\n[Potential Concerns]:\n1. ...\n\n[Prediction & Analysis]:\n...\n"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def get_average_embedding_output(tokens):
    print(type(tokens))
    prompt = f"{B_INST}{B_SYS}{SYSTEM_PROMPT}{E_SYS}{tokens}{E_INST}"
    inputs = tokenizer(
    prompt, return_tensors='pt'
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]  # デコーダの最終隠れ層 (batch, seq_len, hidden_dim)
    # **文埋め込みを取得**
    embedding = last_hidden_state.mean(dim=1)  # 平均プーリング (shape: [batch_size, hidden_dim])
    embedding = embedding.squeeze(0)
    print("type(embedding):", type(embedding))
    print("embedding.shape:", embedding.shape)
    embedding = embedding.tolist()
    return embedding

output = get_average_embedding_output(recommendations[0])
print(output)

<class 'str'>
type(embedding): <class 'torch.Tensor'>
embedding.shape: torch.Size([4096])
[-0.78564453125, -0.390869140625, -0.67041015625, -0.470947265625, 0.70068359375, 0.01062774658203125, 1.328125, -1.1484375, -0.1348876953125, 0.68701171875, 0.412353515625, -0.64111328125, -0.1900634765625, -1.4091796875, -0.4013671875, -0.006237030029296875, 0.0506591796875, -0.97021484375, 0.308837890625, -0.058746337890625, 1.8916015625, 0.3759765625, 0.491455078125, -1.138671875, -0.76416015625, -0.7353515625, 0.9921875, -0.346435546875, -0.515625, -1.537109375, 1.6669921875, -1.1328125, -1.0712890625, 1.935546875, 0.150634765625, -3.255859375, 1.4638671875, 1.4677734375, 0.251220703125, -0.93017578125, -0.671875, -0.52294921875, 0.8310546875, 1.53125, 1.5556640625, 1.1875, -1.412109375, -0.59375, -0.7294921875, -0.2247314453125, 1.583984375, 1.7158203125, 0.603515625, -0.97265625, 2.177734375, 0.50439453125, -0.5224609375, -1.7919921875, 0.1761474609375, -0.018951416015625, -0.21875, -1.6230

In [4]:
print(type(output))
print(len(output))

<class 'list'>
4096


In [8]:
import pickle
import re
import os

load_path = "/home/fukuda/M1_reserch/datasets/pickles_0tumeru_byfingpt-forcaster/for_test/S_test_data_sorted_in_jsons_6_2015-10-01~2015-10-03_20_7_5_4096.pkl"
with open(load_path, "rb") as file:
    loaded_data = pickle.load(file)

In [ ]:
print(len(loaded_data))
print((loaded_data[0]["embedding"].shape))
print((loaded_data[0]["embedding"]))
# print(temp["embedding"].shape)

In [ ]:
import torch
i=0
loaded_data_copy = loaded_data
for loaded_datam in loaded_data_copy:
        loaded_datam["embedding"][:10, :, :, :] = torch.tensor(get_average_embedding_output(recommendations[i]))
        loaded_datam["embedding"][10:, :, :, :] = torch.tensor(get_average_embedding_output(warnings[i]))
        i+=1

In [ ]:
print((loaded_data_copy[0]["embedding"]))
print((loaded_data_copy[1]["embedding"]))

In [10]:
save_path = os.path.join("/home/fukuda/M1_reserch/datasets/dummy_byfingpt-forcaster_0402", os.path.basename(load_path))
print(save_path)
with open(save_path, 'wb') as file:
    pickle.dump(loaded_data, file)

/home/fukuda/M1_reserch/datasets/dummy_byfingpt-forcaster_0402/S_test_data_sorted_in_jsons_6_2015-10-01~2015-10-03_20_7_5_4096.pkl


In [7]:
#次元削減を入れる　fingptのpickleでは4096次元で，訓練した結果，売買をしなくなるため　384次元に削減
import pickle

# pickleファイルのパス
file_path = "/home/fukuda/M1_reserch/datasets/dummy_byfingpt-forcaster_0402/S_test_data_sorted_in_jsons_6_2015-10-01~2015-10-03_20_7_5_4096.pkl"
# pickleファイルを開く
with open(file_path, "rb") as f:
    data = pickle.load(f)  # データを読み込む

In [8]:
print(data[0]["embedding"].shape)
print(type(data[0]["embedding"]))

torch.Size([20, 7, 5, 384])
<class 'torch.Tensor'>


In [3]:
import torch
import torch.nn as nn

x = torch.randn(20, 7, 5, 4096)
linear = nn.Linear(4096, 384)  # 4096次元→384次元への変換
x_reduced = linear(x)

print(x_reduced.shape)  # torch.Size([20, 7, 5, 384])


torch.Size([20, 7, 5, 384])


In [4]:
for datam in data:
    datam["embedding"] = linear(datam["embedding"])

In [6]:
import os
basename = os.path.basename(file_path)
save_folder = "/home/fukuda/M1_reserch/datasets/dummy_byfingpt-forcaster_0402/"
save_dir = save_folder + "/" + basename
print(save_dir)
with open(save_dir, "wb") as f:
    pickle.dump(data, f)


/home/fukuda/M1_reserch/datasets/dummy_byfingpt-forcaster_0402//S_test_data_sorted_in_jsons_6_2015-10-01~2015-10-03_20_7_5_4096.pkl
